In [4]:
import requests
import json
import time
import pandas as pd

In [5]:
def get_summoners_ids(api_key: str, region: str, division: str, tier: str, page: str, ids: list):
    url = f'https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page={page}&api_key={api_key}'
    profiles = requests.get(url)
    if profiles.status_code != 200:
        print(f'Status code is {profiles.status_code}')
    for profile in profiles.json():
        ids.append(profile['summonerId'])
        
def get_acc(region: str, summ_id: str, api_key: str, accs: str):
    url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summ_id}?api_key={api_key}'
    acc = requests.get(url)
    if acc.status_code != 200:
        print(f'Status code is {acc.status_code}')
    accs.append(acc.json()['puuid'])
    
def get_match(region: str, puuid: str, start: int, count: int, api_key: str, matches: list):
    match = requests.get(f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={start}&count={count}&api_key={api_key}')
    if match.status_code != 200:
        print(f'Status code is {match.status_code}')
    matches.append(match.json())
    
def get_match_info(region: str, match: str, api_key: str, match_infos: list):
    match_info = requests.get(f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match}?api_key={api_key}')
    if match_info.status_code != 200:
        print(f'Status code is {match_info.status_code}')
    match_infos.append(match_info.json())

In [68]:
with open('api.txt', 'r') as file:
    api_key = file.read()
region = 'EUW1'
region_2 = 'europe'
division = 'DIAMOND'
pages = 50
tiers = ['I', 'II', 'III', 'IV']
start = 0
count = 10

In [128]:
ids = []
for tier in tiers:
    for i in range(1, pages):
        get_summoners_ids(api_key, region, division, tier, i, ids)
        time.sleep(1.3)

In [ ]:
ids_df = pd.DataFrame(ids, columns = ['Summoner ID'])
ids_df.to_csv('ids.csv')

In [9]:
ids = pd.read_csv('ids.csv', index_col=0)

In [57]:
accs = []
for summ_id in ids['Summoner ID'][:1000]:
    get_acc(region, summ_id, api_key, accs)
    time.sleep(1.3)

In [60]:
accs_df = pd.DataFrame(accs, columns = ['PUUID'])
accs_df.to_csv('puuids.csv')

In [12]:
accs = pd.read_csv('puuids.csv', index_col=0)

In [17]:
matches = []
for acc in accs['PUUID'][:1000]:
    get_match(region_2, acc, start, count, api_key, matches)
    time.sleep(1.3)

In [18]:
matches = [item for sublist in matches for item in sublist]

In [19]:
matches = list(set(matches))

In [69]:
match_infos = []
for match in matches:
    get_match_info(region_2, match, api_key, match_infos)

Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429
Status code is 429


KeyboardInterrupt: 

In [51]:
filtered_match_infos = [match_info for match_info in match_infos if 'info' in match_info]

In [54]:
red_champions = []
blue_champions = []
red_win = []
blue_win = []
for j in range(len(filtered_match_infos)):
    temp_1 = []
    temp_2 = []
    for i in range(5):
        temp_1.append(filtered_match_infos[j]['info']['participants'][i]['championId'])
        temp_2.append(filtered_match_infos[j]['info']['participants'][i+5]['championId'])
    red_champions.append(temp_1)
    blue_champions.append(temp_2)
    red_win.append(filtered_match_infos[j]['info']['teams'][0]['win'])
    blue_win.append(filtered_match_infos[j]['info']['teams'][1]['win'])

In [55]:
columns = {'Red_Win': red_win, 'Blue_Win': blue_win, 'Red_Champions': red_champions, 'Blue_Champions': blue_champions}
summary = pd.DataFrame(data = columns)
summary
summary.to_csv('dataset2.csv')

In [24]:
matches_df = pd.read_csv('dataset.csv', index_col=0)

In [59]:
summary = pd.concat([summary, matches_df], ignore_index=True)

In [66]:
summary.to_csv('dataset3.csv')